# Generate AROMA outputs from the original workflow for integration testing

In [1]:
import os
import os.path as op
import shutil

import numpy as np
import pandas as pd
import pytest
from nilearn import image, datasets

from aroma.tests.utils import get_tests_resource_path

In [2]:
nilearn_data = datasets.fetch_development_fmri(n_subjects=1, age_group="adult")
in_file = nilearn_data.func[0]
confounds_file = nilearn_data.confounds[0]

In [3]:
# Obtain test path and determine filenames
test_path = get_tests_resource_path()
test_file = op.join(test_path, "smoothed_func.nii.gz")
mc_file = op.join(test_path, "mc.tsv")
out_path = op.join(test_path, "integration_test_ground_truth")

# Smooth data using FWHM of 8mm (twice voxel size)
in_img_smooth = image.smooth_img(in_file, fwhm=8)
in_img_smooth.to_filename(test_file)

# Read confounds
confounds = pd.read_table(confounds_file)

# Extract motion parameters from confounds
mc = confounds[["rot_x", "rot_y", "rot_z", "trans_x", "trans_y", "trans_z"]]
mc.to_csv(mc_file, sep="\t", index=False, header=None)

In [4]:
%%bash -s "$test_file" "$mc_file" "$out_path"

python /Users/tsalo/Documents/tsalo/ICA-AROMA/ICA_AROMA.py \
    -i $1 -mc $2 -o $3 -tr 2. -dim 0 -den both


------------------------------- RUNNING ICA-AROMA ------------------------------- 
--------------- 'ICA-based Automatic Removal Of Motion Artifacts' --------------- 

Step 1) MELODIC
Step 2) Automatic classification of the components
  - registering the spatial maps to MNI
  - extracting the CSF & Edge fraction features
  - extracting the Maximum RP correlation feature
  - extracting the High-frequency content feature
  - classification
Found 1 file(s)
Found 11 head motion-related components in a total of 29 components.
Step 3) Data denoising

----------------------------------- Finished -----------------------------------



In /Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [5]:
# Remove downloaded test data
os.remove(mc_file)
os.remove(test_file)

# Collect certain outputs we care about
ica_path = op.join(out_path, "melodic.ica")
os.rename(
    op.join(ica_path, "eigenvalues_percent"),
    op.join(out_path, "eigenvalues_percent")
)
os.rename(
    op.join(ica_path, "melodic_FTmix"),
    op.join(out_path, "melodic_FTmix")
)
os.rename(
    op.join(ica_path, "melodic_IC.nii.gz"),
    op.join(out_path, "melodic_IC.nii.gz")
)
os.rename(
    op.join(ica_path, "melodic_ICstats"),
    op.join(out_path, "melodic_ICstats")
)
os.rename(
    op.join(ica_path, "melodic_mix"),
    op.join(out_path, "melodic_mix")
)
os.rename(
    op.join(ica_path, "melodic_PPCA"),
    op.join(out_path, "melodic_PPCA")
)
os.rename(
    op.join(ica_path, "melodic_Tmodes"),
    op.join(out_path, "melodic_Tmodes")
)

# Remove MELODIC folder
shutil.rmtree(ica_path)